In [28]:
import netCDF4 as nc
import rasterio as rio
import numpy as np
import urllib.request

In [29]:
urllib.request.urlretrieve('ftp://ftp.cdc.noaa.gov/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc', '/Users/nathansuberi/Downloads/air.2x2.250.mon.anom.comb2.nc')

('/Users/nathansuberi/Downloads/air.2x2.250.mon.anom.comb2.nc',
 <email.message.Message at 0x1188250b8>)

In [33]:
gistemp = nc.Dataset('/Users/nathansuberi/Downloads/air.2x2.250.mon.anom.comb2.nc')

In [32]:
gistemp

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: GISS Surface Temperature Analysis (GISTEMP): 250km smoothing combined land/ocean
    comments: This is the 250km smoothed combined land,ocean version of the dataset on a 2x2 grid. SST dataset used at NASA is now ERSST
    platform: Analysis
    Source: http://data.giss.nasa.gov/gistemp/ source and http://data.giss.nasa.gov/pub/gistemp/ data
    Documentation: http://www.esrl.noaa.gov/psd/data/gridded/data.gistemp.html
    references: http://www.esrl.noaa.gov/psd/data/gridded/data.gistemp.html
    Conventions: COARDS
    history: created at PSD Jun 2009 by CAS. Converted to chunked, deflated non-packed NetCDF4 2014/06
    dataset_title: GISS Surface Temperature Analysis
    dimensions(sizes): lon(180), lat(90), time(1658), nbnds(2)
    variables(dimensions): float32 lat(lat), float32 lon(lon), float64 time(time), float32 air(time,lat,lon)
    groups: 

In [23]:
data = gistemp['air'][0,:,:].data
data2 = np.zeros(data.shape)
data2[:,:90] = data[:,90:]
data2[:,90:] = data[:,:90]


In [26]:
data[:,90:].shape
data[:,:90].shape

(90, 90)

In [15]:
south_lat = -90
north_lat = 90
west_lon = -180
east_lon = 180
# Transformation function
transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, data.shape[1], data.shape[0])
# Profile
profile = {
    'driver':'GTiff',
    'height':data.shape[0],
    'width':data.shape[1],
    'count':1,
    'dtype':rio.float32,
    'crs':'EPSG:4326',
    'transform':transform,
    'compress':'lzw',
    'nodata':9999.
}

In [24]:
with rio.open('tmpfile_shifted.tif', 'w', **profile) as dst:
    dst.write(data2.astype(rio.float32), indexes=1)